In [ ]:
import time
import requests 
import hmac
import json
import hashlib
import numpy as np
from datetime import datetime
import pandas as pd
import json
import os
import threading
import csv
#----------
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
#----------
from IPython.display import clear_output

In [1]:
###############################################################################################
#----------------------------------------biance API-------------------------------------------#
###############################################################################################

In [98]:
import time
import hmac
import requests
import json

class biancefAPI():
    def __init__(self):
        self._api_host = 'https://fapi.binance.com'
        self._api_key = b'1vyg6pC1WY-ODMhSyeHasYBV8ZVrgpAKYEu4uqWZ'
        self._api_secret  = 'qXuNtAcsQfZZEbzHlrm_rdBOtCrGgFHrYjdkshw8'
        self.ts = int(time.time() * 1000)
        self._subaccount_name = ""
        self._session = Session()
        
    def _json_encode(self,data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def _sign(self,data):
        j = self._json_encode(data)
        h = hmac.new(self._api_secret, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()
    
    def _get(self,url,**kwargs): 
        res = requests.get(self._api_host+ url,**kwargs)
        res = json.loads(res.text)
        return res


    
    def _post(self,url,data): 
        try:
            signature = self._sign(data)
            data['sig'] = signature
            res = requests.post(self.API_HOST + url, headers= self.header, data=self._json_encode(data))
            return res
        except:
            return res
        
    def serverTime(self):
        return self._get('/fapi/v1/time')
    
    def exchangeInfo(self):
        return self._get('/fapi/v1/exchangeInfo')
        
    def get_ticker(self,sym):
        return self._get(f'/fapi/v1/ticker/bookTicker?symbol={sym}')
    
    def conData(self,sym,interval,startTime,limit):
        return self._get(f'/fapi/v1/klines?symbol={sym}&interval={interval}&startTime={startTime}&endTime={endTime}&limit={limit}')
    
    def placeOrder(self,sym,side,quantity,price,order_type):
        data={
            'symbol':sym,
            'side':side,
            'type':order_type,
            'quantity':quantity,
            'price':price,
            'timestamp':self.serverTime()
        }
        return self._post(f'/fapi/v1/order',data)
    
    def placeMultipleOrder(self,sym,side,quantity,price,order_type):
        data={
            'symbol':sym,
            'side':side,
            'type':order_type,
            'quantity':quantity,
            'price':price,
            'timestamp':self.serverTime()
        }
        return self._post(f'/fapi/v1/order',data)

In [99]:
api = biancefAPI()

In [100]:
api.get_ticker('XRPUSDT')

{'symbol': 'XRPUSDT',
 'bidPrice': '0.5063',
 'bidQty': '14402.6',
 'askPrice': '0.5064',
 'askQty': '8637.9',
 'time': 1616669303821}

In [101]:
api.get_ticker('XLMUSDT')

{'symbol': 'XLMUSDT',
 'bidPrice': '0.36577',
 'bidQty': '3300',
 'askPrice': '0.36578',
 'askQty': '528',
 'time': 1616669305923}

In [102]:
api.serverTime()

{'serverTime': 1616669309037}

In [104]:
info=api.exchangeInfo()

In [106]:
for i in info['symbols']:
    if 'XRP' in i['symbol']:
        print(i['symbol'])
    if 'XLM' in i['symbol']:
        print(i['symbol'])

XRPUSDT
XLMUSDT


In [87]:
a['symbols']

[{'symbol': 'ETHBTC',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': 8,
  'quoteAsset': 'BTC',
  'quotePrecision': 8,
  'quoteAssetPrecision': 8,
  'baseCommissionPrecision': 8,
  'quoteCommissionPrecision': 8,
  'orderTypes': ['LIMIT',
   'LIMIT_MAKER',
   'MARKET',
   'STOP_LOSS_LIMIT',
   'TAKE_PROFIT_LIMIT'],
  'icebergAllowed': True,
  'ocoAllowed': True,
  'quoteOrderQtyMarketAllowed': True,
  'isSpotTradingAllowed': True,
  'isMarginTradingAllowed': True,
  'filters': [{'filterType': 'PRICE_FILTER',
    'minPrice': '0.00000100',
    'maxPrice': '100000.00000000',
    'tickSize': '0.00000100'},
   {'filterType': 'PERCENT_PRICE',
    'multiplierUp': '5',
    'multiplierDown': '0.2',
    'avgPriceMins': 5},
   {'filterType': 'LOT_SIZE',
    'minQty': '0.00100000',
    'maxQty': '100000.00000000',
    'stepSize': '0.00100000'},
   {'filterType': 'MIN_NOTIONAL',
    'minNotional': '0.00010000',
    'applyToMarket': True,
    'avgPriceMins': 5},
   {'filterType':

In [ ]:
############################################################################################
#------------------------------------  symbol class   -------------------------------------#
############################################################################################

In [ ]:
class symbol():
    def __init__(self,sym,API):
        self.symbol = sym
        self.ticker = {'ask':'','bid':'','askv':'','bidv':''}
        self.his_price={}
        self.API = API
        
    def get_ticker(self):
        try:
            ticker = self.API.get_ticker(self.symbol)
            self.ticker['bid'] = float(ticker['bids'][0][0])
            self.ticker['bidv'] = float(ticker['bids'][0][1])
            self.ticker['ask'] = float(ticker['asks'][0][0])
            self.ticker['askv'] = float(ticker['asks'][0][1])
            return True
        except:
            return False
        
    def getHisPrice(self,tf,nbar):
        price = self.API.historicalPrice(self.symbol,tf,nbar,int(time.time())-(tf*nbar),int(time.time()))
        self.his_price = pd.DataFrame.from_dict(price)     
        
    def ma(self,type_bar):
        return self.his_price[type_bar].mean()

    def refBar(self,type_bar,bar):
        return self.his_price[type_bar][len(self.his_price)-bar]
    
    def atr(self):
        tr=[]
        for i in range(len(self.his_price)):
            tr.append(
                max(
                    (self.his_price['high'][i]-self.his_price['low'][i]),
                    abs(self.his_price['high'][i]-self.his_price['close'][i]),
                    (self.his_price['low'][i]-self.his_price['close'][i])
                )) 
        return np.mean(tr)

In [ ]:
###################################################################################
#---------------------------------  main program  --------------------------------#
###################################################################################

In [ ]:
class main():
    def __init__(self,sym_list,side,lot,margin):
        #self.API = bitkubAPI(API_HOST,API_KEY,API_SECRET)
        self.API = ftxAPI()
        self.order = {}
        self.margin = margin
        self.lot = lot
        self.atr = 1
        self.ticker = {}
        self.fee = 0.0004
        self.zone = 0
        self.timeframe = 1440 #min #day:1440  #5m:5
        self.side = side
        self.symbol = sym_list
        self.system = True
        self.api_connect = True
        
        #time
        self.tm = time.localtime() # get struct_time
        self.time_string = time.strftime("%Y-%m-%d, %H:%M:%S", self.tm)
        self.refSec = 0
        self.timeout = 0
        
        #HisPrice init
        self.sys = list([])
        for i in range(len(sym_list)):self.sys.append(i)
        for i in range(len(self.sys)): 
            self.sys[i] = symbol(sym_list[i],self.API)
        self.hisPrice_init()
            
    ########################### write data ###########################          
    def write_log(self,dict_order): 
        csv_columns=[]
        for i in dict_order.keys():
            csv_columns.append(i)
        if(os.path.isfile('log.csv') == False):
            with open('log.csv', 'w', newline='') as csv_object: 
                writer = csv.DictWriter(csv_object, fieldnames=csv_columns)
                writer.writeheader()
                writer.writerow(dict_order) 
        else:
            with open('log.csv', 'a', newline='') as csv_object: 
                writer = csv.DictWriter(csv_object, fieldnames=csv_columns)
                writer.writerow(dict_order)   
            
    def load_order (self):
        if(os.path.isfile('data.json') == True):
            with open('data.json') as infile:
                self.order = json.load(infile)   
            
    def save_order(self): 
        with open('data.json', 'w') as outfile: 
            json.dump(self.order, outfile)

            
    ########################### getdata ###########################   
    def time_check(self):
        #get_time
        self.tm = time.localtime() # get struct_time
        self.time_string = time.strftime("%Y-%m-%d, %H:%M:%S", self.tm)
        return True
    
    def hisPrice_init(self):
        for i in range(len(self.sys)):
            self.sys[i].getHisPrice(self.timeframe*60,100)
        for i in range(len(self.sys)):
            if i == 0: 
                self.ma = self.sys[i].ma('close')
                self.refBar = self.sys[i].refBar('high',1)
            elif i == 1: 
                self.ma = round(self.ma - self.sys[i].ma('close') , 5)
                self.refBar = round(self.refBar - self.sys[i].refBar('high',1) , 5)
            else:
                print('getHisPrice function error')
        
    def getHisPrice(self):
        if(self.refSec != self.tm and self.tm.tm_sec % self.timeframe  == 0):
            self.refSec = self.tm.tm_sec
            hisdatas = list()
            for i in range(len(self.sys)):
                get_hisdata = threading.Thread(target=self.sys[i].getHisPrice, args=[self.timeframe*60, 100])
                get_hisdata.start()
                hisdatas.append(get_hisdata)
            for i in hisdatas:
                i.join()
                
            for i in range(len(self.sys)):
                if i == 0: 
                    self.ma = self.sys[i].ma('close')
                    self.refBar = self.sys[i].refBar('high',1)
                elif i == 1: 
                    self.ma = round(self.ma - self.sys[i].ma('close') , 5)
                    self.refBar = round(self.refBar - self.sys[i].refBar('high',1) , 5)
                else:
                    print('getHisPrice function error')
    
    ########################### open order ###########################             
    def place_orders_open(self,sym,side,size,price):
        res = self.API.place_orders(sym,side,size,price,'market')
        return{ 'status':'open',
                'open_id' : res['id'],
                'open_date': res['createdAt'],
                'open_price': res['price'],
                'side':side,
                'size': res['size'],
                'sl': '',
                'tp': '',
                'fee':self.fee*(res['price']*res['size']),
                #'fee': 0,
                'order_comment':'',
                }

                        
    #-----long order-----side[i]           
    def long_open_conditon(self):
        #------check short_conditon
        long_conditon = all([  self.ticker['ask'] <= self.ma,
                               abs(self.ticker['ask'] - self.zone) < 0.00005,
                               str(self.zone) not in self.order.keys()
                          ])
        
        if(self.time_check() and long_conditon):
            dict_order=list([])
            for i in range(len(self.sys)):
                price = self.ticker['ask'] if self.side[i] == 'buy' else self.ticker['bid']
                dict_order.append(self.place_orders_open(self.sys[i].symbol,self.side[i],self.lot,price))
            
            print(f'---open long {self.zone}---')
            self.order[f'{self.zone}'] = {}
            for i in range(len(self.sys)):
                self.order[f'{self.zone}'][self.sys[i].symbol] = dict_order[i]
                print(dict_order[i])
            self.save_order()
            print('')
            
    #-----short order-----side[-i]            
    def short_open_conditon(self):
        #------check short_conditon
        short_conditon = all([    self.ticker['ask'] >= self.ma,
                               abs(self.ticker['ask'] - self.zone) < 0.00005,
                               str(self.zone) not in self.order.keys()
                          ])

        if(self.time_check() and short_conditon):
            dict_order=list([])
            for i in range(len(self.sys)):
                price = self.ticker['ask'] if self.side[-i] == 'buy' else self.ticker['bid']
                dict_order.append(self.place_orders_open(self.sys[i].symbol,self.side[-i],self.lot,price))
                                  
            print(f'---open short {self.zone}---')
            self.order[f'{self.zone}'] = {}
            for i in range(len(self.sys)):
                self.order[f'{self.zone}'][self.sys[i].symbol] = dict_order[i]
                print(dict_order[i])
            self.save_order()
            print('')

            
    ########################### close_order ###########################       
    #-----place orders------
    def place_orders_close(self,sym,side,size,price,zone,comment):
        res = self.API.place_orders(sym,side,size,price,'market')
        self.order[f'{zone}'][sym]['status'] = 'close'
        self.order[f'{zone}'][sym]['close_id'] = res["id"]
        self.order[f'{zone}'][sym]['close_date'] = res["createdAt"]
        self.order[f'{zone}'][sym]['close_price'] = res["price"]
        self.order[f'{zone}'][sym]['fee'] = self.order[f'{zone}'][sym]['fee'] + self.fee*(res["price"]*self.order[f'{zone}'][sym]['size'])
        
        #calculate order_profit
        if(self.order[f'{zone}'][sym]['side']=='buy'):
            self.order[f'{zone}'][sym]['order_profit'] = (res["price"] - self.order[f'{zone}'][sym]['open_price']) - (self.order[f'{zone}'][sym]['fee']*2)
        elif(self.order[f'{zone}'][sym]['side']=='sell'):
            self.order[f'{zone}'][sym]['order_profit'] = (self.order[f'{zone}'][sym]['open_price'] - res["price"])  - (self.order[f'{zone}'][sym]['fee']*2)
        else:
            print('error calculate order_profit')
            self.system = False
            
        
        self.order[f'{zone}'][sym]['order_comment'] = f'close {comment}'
        self.order[f'{zone}'][sym]['zone'] = zone
        return self.order[f'{zone}'][sym]
        
    #-----process close_order---------
    #-----CLOSE LONG
    def process_closeOrder(self,zone):
        zone = float(zone)
        if(self.order[f'{zone}'][self.sys[0].symbol]['side'] == 'buy'
          and self.ticker['bid'] >= zone + self.margin):                      
            dict_order=list([])
            zProfit = 0                      
            for i in range(len(self.sys)):
                price = self.ticker['ask'] if self.side[-i] == 'buy' else self.ticker['bid']
                dict_order.append(self.place_orders_close(self.sys[i].symbol,self.side[-i],self.lot,price,zone,price))
                #cal zProfit
                zProfit = zProfit + dict_order[i]['order_profit']
                print(f'---test out long---')
                print(dict_order[i]['order_profit'] )
            print(f'---close long order---')
            for i in range(len(self.sys)):
                dict_order[i]['zone_profit']=zProfit
                self.write_log(dict_order[i])
                print(dict_order[i])
            #SAVE LOG
            del self.order[f'{zone}']
            self.save_order()
            print('')
            
        #-----CLOSE SHORT
        elif(self.order[f'{zone}'][self.sys[0].symbol]['side'] == 'sell' 
           and self.ticker['ask'] <= zone - self.margin):
            for i in range(len(self.sys)):
                price = self.ticker['ask'] if self.side[i] == 'buy' else self.ticker['bid']
                dict_order.append(self.place_orders_close(self.sys[i].symbol,self.side[i],self.lot,price,zone,price))
                #cal zProfit
                zProfit = zProfit + dict_order[i]['order_profit'] 
                print(f'---test out short---')
                print(dict_order[i]['order_profit'] )
            print(f'----close short order---')
            for i in range(len(self.sys)):
                dict_order[i]['zone_profit']=zProfit
                self.write_log(dict_order[i])
                print(dict_order[i])
            del self.order[f'{zone}']
            self.save_order()
            print('')
            
    #-----operation------
    def close_order(self):
        processes = list()
        for zone in self.order.keys():
            process = threading.Thread(target=self.process_closeOrder, args=[zone])
            process.start()
            processes.append(process)
        for process in processes:
            process.join()
            
            
    ######################## initialize ########################       
    def initialize(self):              
        print('a')
    ######################## program control ####################
    def controlPanel(self,timeout):
        self.timeout  = timeout*60 #min to sec
        lastsec=0
        while(self.system):
            if(lastsec != self.tm.tm_sec):
                lastsec = self.tm.tm_sec
                self.timeout -= 1
            if(self.timeout<0):
                self.system = False
                
    ########################### start ###########################   
    def start(self):
        #try:

        try:
            #get_ticker
            self.system = all([self.sys[i].get_ticker() for i in range(len(self.sys))])

            #cal_ticker
            self.ticker['ask']  = round(self.sys[0].ticker['ask'] - self.sys[1].ticker['bid'], 5)
            self.ticker['bid']  = round(self.sys[0].ticker['bid'] - self.sys[1].ticker['ask'], 5)

            #----thread set 
            thr_getHisPrice = threading.Thread(target=self.getHisPrice)
            #----thread start 
            thr_getHisPrice.start()
            self.api_connect=True
        except:
            self.api_connect=False


        if(self.api_connect):
            #cal zone
            self.zone = round((self.ticker['ask'] // self.margin) * self.margin,5)
            
            #----thread set 
            #------open long condition ------
            thr_open_long = threading.Thread(target=self.long_open_conditon)
            #------check short_conditon
            thr_open_short = threading.Thread(target=self.short_open_conditon)
               
            #----thread start 
            thr_open_long.start()
            thr_open_short.start()
            #----thread join 
            thr_open_long.join()
            thr_open_short.join()
            
            #------check close_conditon
            self.close_order()
            
            ask  = self.ticker['ask']
            print(f'{self.sys[0].symbol}/{self.sys[1].symbol} zone:{self.zone} ask:{ask} ma:{self.ma} {self.time_string}  timeout:{self.timeout}  ')#,end='\r')
        else:           print(f'connection failed {self.time_string}                                                                        ')#,end='\r')


In [ ]:
program = main(['XRP-PERP','XLM-PERP'],['buy','sell'],1,0.0005)
program.load_order()
controlPanel = threading.Thread(target=program.controlPanel ,args=[60])
controlPanel.start()

while(program.system):
    clear_output(wait=True)
    print(f'---------------------------------------')
    program.start()
    time.sleep(0)